In [1]:
# data science
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

import youtube.get_youtube_data as get_youtube_data
import machine_learning.embedding as embedding

In [2]:
# Step 1. Get the youtube API key. For now we can just grab 1, but in the future
#         we will need to obtain a list of API keys due to the high quota cost of
#         performing a youtube search.
YOUTUBE_API_KEY = get_youtube_data.get_youtube_api_key()
# YOUTUBE_API_KEYS = get_youtube_data.get_youtube_api_key_list()

In [3]:
# Step 2. Build the youtube client to make API calls.
youtube = get_youtube_data.make_client(YOUTUBE_API_KEY)

In [4]:
# Step 3. Perform a youtube search with a user-specified query.
youtube_df = get_youtube_data.search_youtube(
    youtube,
    query='Patrick Bet David',
    max_vids=15,        # youtube accepts 50 as the max value
    order='relevance',   # default is relevance
    transcripts=True
)

Searching for: Patrick Bet David
Returning 15 results


In [5]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                15 non-null     object
 1   published_at            15 non-null     object
 2   channel_id              15 non-null     object
 3   title                   15 non-null     object
 4   description             15 non-null     object
 5   channel_title           15 non-null     object
 6   thumbnail_default_url   15 non-null     object
 7   thumbnail_medium_url    15 non-null     object
 8   thumbnail_high_url      15 non-null     object
 9   thumbnail_standard_url  15 non-null     object
 10  thumbnail_maxres_url    15 non-null     object
 11  tags                    14 non-null     object
 12  video_duration          15 non-null     object
 13  video_caption           15 non-null     object
 14  video_view_count        15 non-null     object
 15  video_li

In [6]:
youtube_df.sample(5)

,video_id,published_at,channel_id,title,description,channel_title,thumbnail_default_url,thumbnail_medium_url,thumbnail_high_url,thumbnail_standard_url,thumbnail_maxres_url,tags,video_duration,video_caption,video_view_count,video_like_count,video_comment_count,transcript
11,6MKZ5ZS4wn8,2023-07-12T13:00:09Z,UCIHdDJ0tjn_3j-FS7s_X1kQ,U.S. Military Crisis: Worst Recruiting Year Si...,"In this episode, Patrick explains the woke U.S...",Valuetainment,https://i.ytimg.com/vi/6MKZ5ZS4wn8/default.jpg,https://i.ytimg.com/vi/6MKZ5ZS4wn8/mqdefault.jpg,https://i.ytimg.com/vi/6MKZ5ZS4wn8/hqdefault.jpg,https://i.ytimg.com/vi/6MKZ5ZS4wn8/sddefault.jpg,https://i.ytimg.com/vi/6MKZ5ZS4wn8/maxresdefau...,"[Entrepreneur, Entrepreneurs, Entrepreneurship...",1501,false,243054,15374,3142,I got some good news for you some bad news for...
6,ZaXqczhYzZo,2023-03-19T19:00:21Z,UCIHdDJ0tjn_3j-FS7s_X1kQ,The Day I Became A NEW MAN - Emotional Story b...,shorts #short #valuetainment #patrickbetdavid.,Valuetainment,https://i.ytimg.com/vi/ZaXqczhYzZo/default.jpg,https://i.ytimg.com/vi/ZaXqczhYzZo/mqdefault.jpg,https://i.ytimg.com/vi/ZaXqczhYzZo/hqdefault.jpg,https://i.ytimg.com/vi/ZaXqczhYzZo/sddefault.jpg,https://i.ytimg.com/vi/ZaXqczhYzZo/maxresdefau...,"[Entrepreneur, Entrepreneurs, Entrepreneurship...",53,false,272025,12970,153,and I went to UCLA Medical Center and they wer...
1,BOv5wzzsqRY,2023-04-23T13:00:05Z,UCIHdDJ0tjn_3j-FS7s_X1kQ,How Patrick Bet-David Raises His Children,shorts #short #valuetainment #patrickbetdavid.,Valuetainment,https://i.ytimg.com/vi/BOv5wzzsqRY/default.jpg,https://i.ytimg.com/vi/BOv5wzzsqRY/mqdefault.jpg,https://i.ytimg.com/vi/BOv5wzzsqRY/hqdefault.jpg,https://i.ytimg.com/vi/BOv5wzzsqRY/sddefault.jpg,https://i.ytimg.com/vi/BOv5wzzsqRY/maxresdefau...,"[Entrepreneur, Entrepreneurs, Entrepreneurship...",25,false,76936,4783,84,if the kids want to have leverage to negotiate...
5,SKBEsVM3xHQ,2023-05-02T23:14:34Z,UCIHdDJ0tjn_3j-FS7s_X1kQ,Patrick Bet-David Offers Tucker Carlson $100 M...,"In this short clip, Patrick Bet-David offers T...",Valuetainment,https://i.ytimg.com/vi/SKBEsVM3xHQ/default.jpg,https://i.ytimg.com/vi/SKBEsVM3xHQ/mqdefault.jpg,https://i.ytimg.com/vi/SKBEsVM3xHQ/hqdefault.jpg,https://i.ytimg.com/vi/SKBEsVM3xHQ/sddefault.jpg,https://i.ytimg.com/vi/SKBEsVM3xHQ/maxresdefau...,"[tucker carlson, fox news, patrick bet david, ...",916,false,1117948,41553,5984,we made a public offer to Tucker Carlson on th...
14,v1eP7z2DbpY,2023-06-21T12:30:23Z,UC2ePGMF5jsWze1sc0Gftf4w,How to Talk Money with Your Spouse - Patrick B...,Money can be a touchy subject. Especially for ...,Mark Savant,https://i.ytimg.com/vi/v1eP7z2DbpY/default.jpg,https://i.ytimg.com/vi/v1eP7z2DbpY/mqdefault.jpg,https://i.ytimg.com/vi/v1eP7z2DbpY/hqdefault.jpg,https://i.ytimg.com/vi/v1eP7z2DbpY/sddefault.jpg,https://i.ytimg.com/vi/v1eP7z2DbpY/maxresdefau...,"[mark savant, mark savant media, small busines...",530,true,4385,200,34,How do you handle financial discussions with y...


In [7]:
filter_sent = "Politics"
list_of_videos = ["Who'se Really Supporting Russia",
                  "The Perfect Hillary Clinton Analogy",
                  "The Evolution of Alex Jones",\
                  "Patrick Bet David on The Breakfast Club",
                  "The Truth About The 2020 Election",
                  "Kobe Bryant's Last Great Interview"]
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embedding.filter_out_embed(model,filter_sent,list_of_videos)


['Patrick Bet David on The Breakfast Club',
 "Kobe Bryant's Last Great Interview"]

In [8]:
titles = youtube_df['title'].tolist()
print("titles:",titles)
print("api test")
print(embedding.filter_out_embed(model,filter_sent,titles))

titles: ['Why Hollywood Elites Don’t Want You To Watch Sound Of Freedom', 'How Patrick Bet-David Raises His Children', 'Patrick Bet-David&#39;s Top 5 Books - MUST READS for Entrepreneurs!', 'How MrBeast is worth $20 BILLION 🤝', 'Patrick Bet-David&#39;s Multi Millionaire Diet', 'Patrick Bet-David Offers Tucker Carlson $100 Million To Join Valuetainment', 'The Day I Became A NEW MAN - Emotional Story by Patrick Bet-David', '“It’s Such BS!” - People Calling Jonah Hill Abusive For Text To Ex-Girlfriend', 'Dave Rubin | PBD Podcast | Ep. 286', 'Patrick Bet-David Explains Who George Soros is &amp; Why He&#39;s Important in Society', 'Patrick Bet-David Makes Tucker Carlson $100 Million Offer to Join His@VALUETAINMENT Content Company', 'U.S. Military Crisis: Worst Recruiting Year Since 1973', 'Inside Patrick Bet-David&#39;s $25 Million Dream Home', '&quot;American Psycho&quot; - Will Gavin Newsom Be The Next President?', 'How to Talk Money with Your Spouse - Patrick Bet David']
api test
['Why H